
# Tutorial: Head-to-Head: Comparing Multiple Models with Cross-Validation

Selamat datang di subchapter 4.2! Setelah kita menetapkan skor baseline yang harus dikalahkan, sekarang saatnya untuk mengadu beberapa model canggih dalam sebuah "turnamen" yang adil.

**Tujuan:** Kita akan belajar cara menulis skrip yang efisien untuk melatih dan mengevaluasi beberapa model yang berbeda secara sistematis. Kita akan menggunakan **Cross-Validation** untuk memastikan perbandingan kita adil dan andal, bukan hanya berdasarkan "keberuntungan" dari satu kali `train_test_split`.

Di akhir notebook ini, kita akan memiliki daftar peringkat yang jelas tentang model mana yang paling menjanjikan untuk masalah kita.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menyiapkan daftar model kandidat untuk dievaluasi.
* Membuat *loop* untuk melatih dan mengevaluasi setiap model menggunakan `cross_val_score`.
* Menyimpan dan membandingkan hasil performa dari setiap model secara terstruktur.
* Menentukan model-model teratas yang layak untuk dianalisis lebih lanjut.




---
### 2. Setup: Data dan Library

Kita akan memuat semua "kontestan" kita di awal. Ini termasuk model linear sederhana, model regularisasi, dan model ensemble yang kuat.



In [2]:

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Model-model yang akan kita bandingkan
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Muat data
housing = fetch_california_housing()
X, y = housing.data, housing.target

# PENTING: Untuk cross-validation, kita bisa menggunakan seluruh dataset (X, y)
# karena cross_val_score akan menangani pembagiannya secara internal.
# Namun, untuk praktik terbaik, kita tetap akan memisahkan test set untuk evaluasi akhir di subchapter selanjutnya.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



HTTPError: HTTP Error 403: Forbidden


---
### 3. Langkah 1: Siapkan "Kontestan" (Daftar Model)

Cara terbaik untuk mengelola beberapa model adalah dengan menyimpannya dalam sebuah *dictionary* atau *list*. Ini membuat proses evaluasi menjadi mudah untuk diotomatisasi dengan *loop*.



In [ ]:
models = {
	'Linear Regression': make_pipeline(
		StandardScaler(),
		LinearRegression()
	),
	'Ridge': make_pipeline(
		StandardScaler(),
		Ridge()
	),
	'Lasso':make_pipeline(
		StandardScaler(),
		Lasso()
	),
	'Decision Tree':make_pipeline(
		StandardScaler(),
		DecisionTreeRegressor()
	),
	'Random Forest':make_pipeline(
		StandardScaler(),
		RandomForestRegressor()
	),
	'Gradient Boosting':make_pipeline(
		StandardScaler(),
		GradientBoostingRegressor()
	),
}


---
### 4. Langkah 2: Gelar "Pertandingan" (Loop dan Evaluasi)

Sekarang kita akan membuat *loop* yang akan menjalankan 5-fold cross-validation pada setiap model di dalam *dictionary* `models`. Kita akan menghitung skor R² dan menyimpan hasilnya.



In [ ]:
results = {}

print("Memulai evaluasi model...")

for name, model in models.items():
	# folding 5 cross validation
	scores = cross_val_score(model,X_train,y_train, cv=5, scoring='r2',n_jobs=-1)

	# simpan hasil
	results[name] = {
		'mean_r2':np.mean(scores),
		'std_r2':np.std(scores)
	}
	print(f"Evaluasi untuk {name} selesai.")

Memulai evaluasi model...
Evaluasi untuk Linear Regression selesai.
Evaluasi untuk Ridge selesai.
Evaluasi untuk Lasso selesai.
Evaluasi untuk Decision Tree selesai.
Evaluasi untuk Random Forest selesai.
Evaluasi untuk Gradient Boosting selesai.



---
### 5. Langkah 3: Umumkan "Pemenang" (Analisis Hasil)

Setelah *loop* selesai, kita bisa mengubah hasilnya menjadi DataFrame Pandas untuk analisis yang mudah.



In [ ]:
results_df = pd.DataFrame.from_dict(results, orient='index')

results_df = results_df.sort_values(by='mean_r2', ascending=False)

print("\n---- Hasil Perbandingan Model ----")
print(results_df)


---- Hasil Perbandingan Model ----
                    mean_r2    std_r2
Random Forest      0.804089  0.006002
Gradient Boosting  0.786588  0.003194
Linear Regression  0.611484  0.006467
Ridge              0.611484  0.006460
Decision Tree      0.607226  0.019406
Lasso             -0.000317  0.000428



**Analisis Hasil:**

Dari tabel di atas, kita dapat melihat peringkat performa model-model kita berdasarkan skor R² rata-rata dari 5-fold cross-validation:

1.  **Gradient Boosting** dan **Random Forest** jelas merupakan dua model teratas, dengan performa yang sangat mirip dan kuat (R² mendekati 0.8).
2.  **Ridge** dan **Linear Regression** berada di tingkat berikutnya, menunjukkan performa yang layak tetapi sedikit di bawah model-model ensemble.
3.  **Lasso** memiliki performa yang sedikit lebih buruk, kemungkinan karena ia menolkan terlalu banyak koefisien.
4.  **Decision Tree** tunggal, seperti yang kita duga, memiliki performa yang paling rendah di antara model-model non-linear karena kecenderungannya untuk *overfitting*.

Perhatikan juga kolom `std_r2`. Model dengan `std_r2` yang lebih rendah lebih **stabil** atau konsisten performanya di berbagai subset data.




---
### 6. Kesimpulan Tutorial

Dengan beberapa baris kode dan sebuah *loop*, kita telah berhasil mengevaluasi 6 model berbeda secara adil dan sistematis. Kita sekarang memiliki bukti kuat bahwa **Gradient Boosting** dan **Random Forest** adalah kandidat terkuat kita.

Di subchapter selanjutnya, kita akan belajar cara **memvisualisasikan** hasil ini menggunakan *box plot* untuk mendapatkan pemahaman yang lebih intuitif tentang performa dan stabilitas setiap model.
